Работал локально, через Google Drive, где лежал архив

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/data_test_short.zip -d video_folder

Archive:  /content/drive/MyDrive/data_test_short.zip
   creating: video_folder/data_test_short/
   creating: video_folder/data_test_short/-220020068_456249220/
  inflating: video_folder/data_test_short/-220020068_456249220/-220020068_456249220.mp4  
   creating: video_folder/data_test_short/-220020068_456249373/
  inflating: video_folder/data_test_short/-220020068_456249373/-220020068_456249373.mp4  
   creating: video_folder/data_test_short/-220020068_456249231/
  inflating: video_folder/data_test_short/-220020068_456249231/-220020068_456249231.mp4  
   creating: video_folder/data_test_short/-220020068_456255339/
  inflating: video_folder/data_test_short/-220020068_456255339/-220020068_456255339.mp4  
   creating: video_folder/data_test_short/-220020068_456249284/
  inflating: video_folder/data_test_short/-220020068_456249284/-220020068_456249284.mp4  
   creating: video_folder/data_test_short/-220020068_456241671/
  inflating: video_folder/data_test_short/-220020068_456241671/-220020

In [3]:
import json

with open('/content/drive/My Drive/test_labels.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [4]:
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

In [5]:
for key, value in test_data.items():
    value['start_sec'] = time_to_seconds(value['start'])
    value['end_sec'] = time_to_seconds(value['end'])
    value['duration'] = value['end_sec'] - value['start_sec']

In [13]:
potential_openings_test = {
    key: value for key, value in test_data.items()
    if value['start_sec'] <= 300 and 0 <= value['duration'] <= 60
}
print("Potential openings in test:", len(potential_openings_test))

Potential openings in test: 40


In [14]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [1]:
import os
!mkdir -p frames
for folder in os.listdir('video_folder/data_test_short'):
    video_path = os.path.join('video_folder/data_test_short', folder, f"{folder}.mp4")
    if os.path.exists(video_path):
        !ffmpeg -i {video_path} -vf "select='lt(t,300)',setpts=N/FRAME_RATE/TB" -vframes 20 frames/{folder}_frame_%d.png

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [2]:
!mkdir -p audio
for folder in os.listdir('video_folder/data_test_short'):
    video_path = os.path.join('video_folder/data_test_short', folder, f"{folder}.mp4")
    if os.path.exists(video_path):
        !ffmpeg -i {video_path} -vn -acodec mp3 -t 300 audio/{folder}.mp3

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [3]:
with open('/content/drive/My Drive/mapping.txt', 'w') as f:
    f.write("ID,Video_Path,Audio_Path\n")
    for folder in os.listdir('video_folder/data_test_short'):
        video_path = os.path.join('video_folder/data_test_short', folder, f"{folder}.mp4")
        if os.path.exists(video_path):
            f.write(f"{folder},{video_path},audio/{folder}.mp3\n")

In [8]:
import json
import librosa
import numpy as np
!mkdir -p audio_features

for folder in os.listdir('video_folder/data_test_short'):
    audio_path = os.path.join('audio', f"{folder}.mp3")
    if os.path.exists(audio_path):
        y, sr = librosa.load(audio_path, sr=None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=1024).tolist()
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=1024).tolist()
        features = {
            'mfcc': mfcc,
            'spectral_contrast': contrast
        }
        with open(f'audio_features/{folder}.json', 'w') as f:
            json.dump(features, f, indent=2)